In [183]:
import pandas as pd
# nhập dữ liệu
fulldata_train = pd.read_csv('https://raw.githubusercontent.com/NHANGIOI/Machine_Learning/main/home-data-for-ml-course/train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/NHANGIOI/Machine_Learning/main/home-data-for-ml-course/test.csv')
Id_test = x_test['Id'].copy()


In [184]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(fulldata_train.drop(columns = ['SalePrice']),fulldata_train['SalePrice'],
                                                                              train_size = 0.8, test_size = 0.2,random_state = 1)

col_missing = [col for col in x_train.columns if x_train[col].isnull().any()]

x_train.drop(col_missing,axis = 1,inplace = True)
x_valid.drop(col_missing,axis = 1,inplace = True)
x_test.drop(col_missing, axis = 1,inplace = True)
# Get list of categorical variables

s = (x_train.dtypes == 'object')
obj_cols = list(s[s].index)

In [185]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def Tree_model(x_train, y_train, x_valid, y_valid, x_test,maxx : int, out_ans : bool):
    model = DecisionTreeRegressor(max_leaf_nodes = maxx,random_state = 1)
    model.fit(x_train,y_train)
    if(out_ans == False):
        MAE = mean_absolute_error(y_valid,model.predict(x_valid))
        return MAE
    else:
        return pd.DataFrame({
            'Id' : Id_test,
            'SalePrice' : model.predict(x_test)
        })
def Forest_model(x_train, y_train, x_valid, y_valid,x_test,out_ans : bool):
    model = RandomForestRegressor(n_estimators = 100, max_depth = 15, min_samples_leaf = 2, n_jobs = -1,random_state = 1)
    model.fit(x_train,y_train)
    if (out_ans == False):
        MAE = mean_absolute_error(y_valid,model.predict(x_valid))
        return MAE
    else:
        return pd.DataFrame({
            'Id' : Id_test,
            'SalePrice' : model.predict(x_test)
        })

In [186]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
class pre_process:
    def drop_col(x_train,x_valid,x_test):
        drop_col_train = x_train.copy().select_dtypes(exclude = ['object'])
        drop_col_valid = x_valid.copy().select_dtypes(exclude = ['object'])
        drop_col_test = x_test.copy().select_dtypes(exclude = ['object'])
        return drop_col_train, drop_col_valid, drop_col_test
    def Ordinal_encode(train,valid,test):
        my_encoder = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value = -1)
        
        label_train = train.copy()
        label_valid = valid.copy()
        label_test = test.copy()
        
        label_train[obj_cols] = my_encoder.fit_transform(train[obj_cols])
        label_valid[obj_cols]  = my_encoder.transform(valid[obj_cols])
        label_test[obj_cols] = my_encoder.transform(test[obj_cols])
        return label_train, label_valid, label_test

In [187]:
x_train, x_valid, x_test = pre_process.Ordinal_encode(x_train,x_valid,x_test)

x_test_ = x_test.copy().reindex(columns=x_train.columns, fill_value=0)

#Forest_model(x_train,y_train,x_valid,y_valid,x_test,True).to_csv("/content/submission.csv",index = False)

minn = 1e9 + 7
opt_leaf = 0
for i in range(3,1000):
    res = Tree_model(x_train, y_train, x_valid, y_valid, x_test,i,False)
    if(minn > res):
        minn = res
        opt_leaf = i
Tree_model(x_train, y_train, x_valid, y_valid, x_test, opt_leaf,True).to_csv("/content/submission.csv",index = False)